In [1]:
#import libraries
import pandas as pd
import numpy as np
import warnings
import pyodbc
import timeit
from datetime import datetime
from datetime import timedelta
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import os
import shutil
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime
from datetime import date
import math
import statistics
import scipy.stats

In [2]:
PT_Data = pd.read_csv('Group 6 Data.csv')

In [3]:
PT_Data.shape

(1615175, 8)

In [4]:
PT_Data.columns

Index(['US News Ranking', 'Hospital Name', 'Billing Code Type', 'DRG Number',
       'DRG Description', 'Payor', 'Plan', 'Charge Amount'],
      dtype='object')

In [5]:
PT_Data

,US News Ranking,Hospital Name,Billing Code Type,DRG Number,DRG Description,Payor,Plan,Charge Amount
0,8,NYU Langone Hospitals Hospital A,MSDRG,1,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,Gross Charge,Hospital A-C Gross Charges,NaN
1,8,NYU Langone Hospitals Hospital A,MSDRG,2,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,Gross Charge,Hospital A-C Gross Charges,NaN
2,8,NYU Langone Hospitals Hospital A,MSDRG,3,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,Gross Charge,Hospital A-C Gross Charges,NaN
3,8,NYU Langone Hospitals Hospital A,MSDRG,4,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,Gross Charge,Hospital A-C Gross Charges,NaN
4,8,NYU Langone Hospitals Hospital A,MSDRG,5,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,Gross Charge,Hospital A-C Gross Charges,NaN
...,...,...,...,...,...,...,...,...
1615170,17,Barnes-Jewish Hospital,DRG,982,extensive o.r. procedures unrelated to princip...,Other,ACCESS PPO,48547.19
1615171,17,Barnes-Jewish Hospital,DRG,982,extensive o.r. procedures unrelated to princip...,United Healthcare,UHC,12816.00
1615172,17,Barnes-Jewish Hospital,DRG,982,extensive o.r. procedures unrelated to princip...,United Healthcare,UHC MEDICARE,19367.35
1615173,17,Barnes-Jewish Hospital,DRG,983,extensive o.r. procedures unrelated to princip...,Other,ACCESS PPO,31154.50


In [6]:
PT_Data.groupby("DRG Number")["Charge Amount"].mean()

DRG Number
1      659043.724513
2      360220.579451
3      492170.675988
4      330071.518721
5      320765.857177
           ...      
987     81984.559476
988     37833.433733
989     24917.426809
998     17551.601134
999      3282.092908
Name: Charge Amount, Length: 867, dtype: float64

In [7]:
PT_Data.groupby("DRG Number")["Charge Amount"].describe()

,count,mean,std,min,25%,50%,75%,max
DRG Number,,,,,,,,
1,1675.0,659043.724513,980834.095924,0.0,270643.50,576211.20,801935.80,36495200.00
2,1638.0,360220.579451,369156.278318,0.0,130867.80,291542.70,413099.90,5386266.00
3,1700.0,492170.675988,432025.919220,0.0,179122.00,376072.70,493533.30,2417894.00
4,1689.0,330071.518721,301723.584890,0.0,117552.00,254962.70,371683.00,1893481.00
5,1594.0,320765.857177,361981.875771,0.0,96346.06,203973.30,313069.80,1619772.00
...,...,...,...,...,...,...,...,...
987,1678.0,81984.559476,74042.453706,0.0,31406.54,63798.15,84702.65,426353.80
988,1669.0,37833.433733,27762.220791,0.0,16308.12,33232.14,44079.72,193921.10
989,1664.0,24917.426809,18020.946972,0.0,11322.13,21882.11,30214.73,97527.48


In [8]:
PT_Data.groupby("Payor")["Charge Amount"].describe()

,count,mean,std,min,25%,50%,75%,max
Payor,,,,,,,,
AETNA,8445.0,37922.740331,43576.191393,33.00,16932.1700,27070.460,42070.4700,735033.1
AETNA,281.0,29473.166014,22671.026262,103.00,14053.0000,25161.000,38376.0000,222752.0
Aetna,56462.0,46949.209029,76316.669503,0.00,16115.7800,27417.710,49227.1600,2053177.0
Aetna,768.0,36052.602891,36864.360345,2910.27,15990.4350,24778.530,39576.9775,321289.1
BCBS,198925.0,47349.361696,61184.743559,0.00,17505.7000,31320.620,54228.6700,1303638.0
BCBS,1.0,24191.000000,NaN,24191.00,24191.0000,24191.000,24191.0000,24191.0
CIGNA,78.0,45522.256410,34856.266061,2990.00,18142.7500,36594.000,61942.0000,175687.0
CIGNA,7965.0,28930.993691,35428.249848,120.00,12515.4600,19211.400,32708.2400,1062342.0
Cigna,58931.0,47649.349554,64643.668338,0.00,18264.3700,32143.610,53978.2000,1759353.0


In [16]:
PT_Data.groupby("US News Ranking")["Hospital Name"].describe()

,count,unique,top,freq
US News Ranking,,,,
1,8507,1,Mayo Clinic Rochester,8507
2,60312,1,Cleveland Clinic,60312
3,739,1,UCLA Medical Center,739
5,47488,1,Massachusetts General,47488
7,24948,1,New York-Presbyterian Hospital,24948
8,1240239,8,NYU Langone Hospitals Hospital D,312169
10,11436,1,Northwestern Memorial Hospital,11436
11,97920,1,University of Michigan,97920
13,6300,1,Hospitals of the University of Pennsylvania-Pe...,6300


In [10]:
PT_Data.describe(include='all')

,US News Ranking,Hospital Name,Billing Code Type,DRG Number,DRG Description,Payor,Plan,Charge Amount
count,1.615175e+06,1615175,1615175,1.615175e+06,1614917,1615175,1615175,1.259907e+06
unique,NaN,22,5,NaN,3284,16,2364,NaN
top,NaN,NYU Langone Hospitals Hospital D,MSDRG,NaN,HIP AND FEMUR PROCEDURES EXCEPT MAJOR JOINT WI...,Other,Gross Charge,NaN
freq,NaN,312169,1265187,NaN,1757,1106737,4002,NaN
mean,8.482837e+00,NaN,NaN,4.792541e+02,NaN,NaN,NaN,5.110536e+04
std,2.891119e+00,NaN,NaN,2.808202e+02,NaN,NaN,NaN,9.118900e+04
min,1.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN,NaN,0.000000e+00
25%,8.000000e+00,NaN,NaN,2.420000e+02,NaN,NaN,NaN,1.426800e+04
50%,8.000000e+00,NaN,NaN,4.750000e+02,NaN,NaN,NaN,2.781052e+04
75%,8.000000e+00,NaN,NaN,7.230000e+02,NaN,NaN,NaN,5.592675e+04
